# **simple numeric full-state EKF example**

I'll use a **2D state** (position + velocity) and a **position-only measurement**. This is linear, so it behaves like a KF, but it's still a correct EKF instance and keeps the arithmetic clean.

---

## 0) Notation match (my usual vs your screenshot)

**Your screenshot**

* Prediction:
  $$
  \hat x_k = g(\hat x_{k-1}, u_k),\qquad
  \hat P_k = G_k P_{k-1} G_k^T + Q_k
  $$
* Update:
  $$
  K = \hat P_k C_k^T (C_k \hat P_k C_k^T + R_k)^{-1}
  $$
  $$
  \hat x'_k = \hat x_k + K\big(z_k - C(\hat x_k)\big)
  $$
  $$
  \hat P'_k = \hat P_k - K C_k \hat P_k
  $$

**Equivalent "standard EKF" naming**

* $g \leftrightarrow f$ (process model)
* $G_k \leftrightarrow F_k = \frac{\partial g}{\partial x}$ (process Jacobian)
* $C(\cdot) \leftrightarrow h(\cdot)$ (measurement model)
* $C_k \leftrightarrow H_k = \frac{\partial C}{\partial x}$ (measurement Jacobian)

✅ Full-state EKF means $\hat x_k$ is the *actual state estimate*, not an error-state.

---

## 1) Define the system (this is the "attached notation" version)

### State

$$
x_k = \begin{bmatrix} p_k \\ v_k \end{bmatrix} \quad\text{(position, velocity)}
$$

### Input (acceleration command)

$$
u_k = a_k
$$

### Process model $g(\cdot)$ with $\Delta t = 1$

Constant-acceleration discrete dynamics:
$$
g(x_{k-1}, u_k)= \begin{bmatrix} p_{k-1} + v_{k-1}\Delta t + \tfrac12 a_k \Delta t^2 \\ v_{k-1} + a_k\Delta t \end{bmatrix}
$$
With $\Delta t=1$:
$$
g(x_{k-1}, a_k)= \begin{bmatrix} p_{k-1} + v_{k-1} + \tfrac12 a_k \\ v_{k-1} + a_k \end{bmatrix}
$$

### Process Jacobian $G_k = \frac{\partial g}{\partial x}$

$$
G_k = \begin{bmatrix} 1 & \Delta t \\ 0 & 1 \end{bmatrix} = \begin{bmatrix} 1 & 1 \\ 0 & 1 \end{bmatrix}
$$

### Process noise covariance

$$
Q_k = \begin{bmatrix} 0.2 & 0 \\ 0 & 0.1 \end{bmatrix}
$$

### Measurement model: position-only

$$
z_k = C(x_k) + v_k,\qquad C(x)= [1\ \ 0]x = p
$$
So:
$$
C(x_k)=p_k
$$
Measurement Jacobian:
$$
C_k = \frac{\partial C}{\partial x} = \begin{bmatrix}1 & 0\end{bmatrix}
$$
Measurement noise covariance:
$$
R_k = 0.5
$$

---

## 2) Plug in numbers

### Given at time $k-1$

$$
\hat x_{k-1} = \begin{bmatrix} 0 \\ 1 \end{bmatrix}, \qquad P_{k-1} = \begin{bmatrix} 1 & 0 \\ 0 & 1 \end{bmatrix}
$$
Input:
$$
u_k=a_k=2
$$
Measurement:
$$
z_k = 2.6
$$

---

## 3) Prediction step (your screenshot's top block)

### 3.1 State prediction

$$
\hat x_k = g(\hat x_{k-1}, u_k) = \begin{bmatrix} 0 + 1 + \tfrac12(2) \\ 1 + 2 \end{bmatrix} = \begin{bmatrix} 2 \\ 3 \end{bmatrix}
$$

### 3.2 Covariance prediction

$$
\hat P_k = G_k P_{k-1} G_k^T + Q_k
$$
First compute $G_k P_{k-1} G_k^T = G_k G_k^T$ since $P_{k-1}=I$:
$$
G_k G_k^T = \begin{bmatrix} 1 & 1 \\ 0 & 1 \end{bmatrix} \begin{bmatrix} 1 & 0 \\ 1 & 1 \end{bmatrix} = \begin{bmatrix} 2 & 1 \\ 1 & 1 \end{bmatrix}
$$
Add $Q_k$:
$$
\hat P_k = \begin{bmatrix} 2 & 1 \\ 1 & 1 \end{bmatrix} + \begin{bmatrix} 0.2 & 0 \\ 0 & 0.1 \end{bmatrix} = \begin{bmatrix} 2.2 & 1 \\ 1 & 1.1 \end{bmatrix}
$$

---

## 4) Update step (your screenshot's bottom block)

### 4.1 Innovation (residual)

Predicted measurement:
$$
C(\hat x_k)= [1\ \ 0]\begin{bmatrix}2\\3\end{bmatrix}=2
$$
Residual:
$$
r_k = z_k - C(\hat x_k)=2.6-2=0.6
$$

### 4.2 Kalman gain

$$
K = \hat P_k C_k^T (C_k \hat P_k C_k^T + R_k)^{-1}
$$
Compute:
$$
S = C_k \hat P_k C_k^T + R_k
$$
Since $C_k=[1\ 0]$, this picks $\hat P_{11}$:
$$
S = 2.2 + 0.5 = 2.7
$$
Now:
$$
\hat P_k C_k^T = \begin{bmatrix} 2.2 & 1 \\ 1 & 1.1 \end{bmatrix} \begin{bmatrix} 1 \\ 0 \end{bmatrix} = \begin{bmatrix} 2.2 \\ 1 \end{bmatrix}
$$
So:
$$
K = \frac{1}{2.7} \begin{bmatrix} 2.2 \\ 1 \end{bmatrix} = \begin{bmatrix} 0.814814 \\ 0.370370 \end{bmatrix}
$$

### 4.3 State update

$$
\hat x'_k = \hat x_k + K r_k = \begin{bmatrix} 2 \\ 3 \end{bmatrix} + \begin{bmatrix} 0.814814 \\ 0.370370 \end{bmatrix} (0.6)
$$
$$
\hat x'_k = \begin{bmatrix} 2 + 0.488888 \\ 3 + 0.222222 \end{bmatrix} = \begin{bmatrix} 2.488888 \\ 3.222222 \end{bmatrix}
$$

### 4.4 Covariance update (exactly as in your screenshot)

$$
\hat P'_k = \hat P_k - K C_k \hat P_k
$$
Result:
$$
\hat P'_k \approx \begin{bmatrix} 0.407407 & 0.185185 \\ 0.185185 & 0.729630 \end{bmatrix}
$$

(You'll notice it stays symmetric here; in practice many people use the **Joseph form** to guarantee symmetry numerically.)

---

## Final numeric result (one EKF step)

* Predicted:
  $$
  \hat x_k = \begin{bmatrix}2\\3\end{bmatrix},\quad
  \hat P_k = \begin{bmatrix}2.2&1\\1&1.1\end{bmatrix}
  $$
* Updated with $z_k=2.6$:
  $$
  \hat x'_k = \begin{bmatrix}2.488888\\3.222222\end{bmatrix},\quad
  \hat P'_k \approx \begin{bmatrix} 0.407407 & 0.185185 \\ 0.185185 & 0.729630 \end{bmatrix}
  $$

---

If you want, I can do the **same exact example but truly "EKF" nonlinear** (e.g., $C(x)=p^2$ or range-to-landmark), where $C_k$ changes with $\hat x_k$.